In [155]:
import pandas as pd
import numpy as np
import re

In [156]:
data=pd.read_csv(r"C:\Users\hp\Downloads\0000000000002747_training_twitter_x_y_train.csv",delimiter=",")
test=pd.read_csv(r"C:\Users\hp\Downloads\0000000000002747_test_twitter_x_test.csv",delimiter=",")

In [157]:
data.head()

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)


In [158]:
test.head()

,tweet_id,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,569682010270101504,American,NaN,zsalim03,NaN,0,@AmericanAir In car gng to DFW. Pulled over 1h...,NaN,2015-02-22 18:15:50 -0800,Texas,Central Time (US & Canada)
1,569608307184242688,American,NaN,sa_craig,NaN,0,"@AmericanAir after all, the plane didn’t land ...",NaN,2015-02-22 13:22:57 -0800,"College Station, TX",Central Time (US & Canada)
2,567879304593408001,Southwest,NaN,DanaChristos,NaN,1,@SouthwestAir can't believe how many paying cu...,NaN,2015-02-17 18:52:31 -0800,CT,Eastern Time (US & Canada)
3,569757651539660801,US Airways,NaN,rossj987,NaN,0,@USAirways I can legitimately say that I would...,NaN,2015-02-22 23:16:24 -0800,"Washington, D.C.",Eastern Time (US & Canada)
4,569900705852608513,American,NaN,tranpham18,NaN,0,@AmericanAir still no response from AA. great ...,NaN,2015-02-23 08:44:51 -0800,New York City,Eastern Time (US & Canada)


In [159]:
df=data.copy()
xtest=test.copy()
df["text"].isna().sum()

0

In [160]:
df=df.drop('name',axis=1)
df=df.drop('tweet_id',axis=1)

del df ["airline_sentiment_gold"]
del df ["negativereason_gold"]
del df["retweet_count"]
del df["tweet_coord"]
del df["tweet_created"]
del df["tweet_location"]
del df["user_timezone"]
xtest=xtest.drop('name',axis=1)
xtest=xtest.drop('tweet_id',axis=1)
del xtest["airline_sentiment_gold"]
del xtest["negativereason_gold"]
del xtest["retweet_count"]
del xtest["tweet_coord"]
del xtest["tweet_created"]
del xtest["tweet_location"]
del xtest["user_timezone"]
del df["airline"]
del xtest["airline"]
df.head()

,airline_sentiment,text
0,negative,"@SouthwestAir I am scheduled for the morning, ..."
1,positive,@SouthwestAir seeing your workers time in and ...
2,positive,@united Flew ORD to Miami and back and had gr...
3,negative,@SouthwestAir @dultch97 that's horse radish 😤🐴
4,negative,@united so our flight into ORD was delayed bec...


In [161]:
df['airline_sentiment'].value_counts()/len(df)




negative    0.623953
neutral     0.211931
positive    0.164117
Name: airline_sentiment, dtype: float64

In [162]:


df["text"].isna().sum()

0

In [163]:
print('Number of observations are: ',len(df))
datasets=df.text
datasets[0:11]


Number of observations are:  10980


0     @SouthwestAir I am scheduled for the morning, ...
1     @SouthwestAir seeing your workers time in and ...
2     @united Flew ORD to Miami and back and  had gr...
3        @SouthwestAir @dultch97 that's horse radish 😤🐴
4     @united so our flight into ORD was delayed bec...
5     @united Why did you load us in this flying sar...
6     @JetBlue that is a stock response. Delays not ...
7     @JetBlue That'd be nice! Hoping to rack up eno...
8     @united frankly worse customer service ever. P...
9     @SouthwestAir yeah haha. Never been in one. It...
10    @SouthwestAir MCO-&gt;DCA flight almost full o...
Name: text, dtype: object

In [164]:
datasets.dtype

dtype('O')

In [165]:
data_lower=[]
for upper in datasets:
    data_lower.append(upper.lower())
data_lower   

['@southwestair i am scheduled for the morning, 2 days after the fact, yes..not sure why my evening flight was the only one cancelled flightled',
 '@southwestair seeing your workers time in and time out going above and beyond is why i love flying with you guys. thank you!',
 '@united flew ord to miami and back and  had great crew, service on both legs. thanks',
 "@southwestair @dultch97 that's horse radish 😤🐴",
 '@united so our flight into ord was delayed because of air force one, but the last flight to sbn is at 8:20, 5 mins from now we just landed.',
 '@united why did you load us in this flying sardine can if you knew the pilots were 2 hours late flight?? #incompetent beyond belief',
 "@jetblue that is a stock response. delays not as frustrating as poor cust serv &amp; being told by 3 ppl to wait &amp; they'd come back but did not.",
 "@jetblue that'd be nice! hoping to rack up enough miles to take a trip to seattle and enjoy a perfect latte in the city of coffee.",
 '@united frankly

In [166]:
data = np.array(data_lower)
print(type(data))

ytrain = df['airline_sentiment'].values
print(data_lower[0:5])
print("ytrain" ,ytrain[0:5])

#X = data[:,1]

<class 'numpy.ndarray'>
['@southwestair i am scheduled for the morning, 2 days after the fact, yes..not sure why my evening flight was the only one cancelled flightled', '@southwestair seeing your workers time in and time out going above and beyond is why i love flying with you guys. thank you!', '@united flew ord to miami and back and  had great crew, service on both legs. thanks', "@southwestair @dultch97 that's horse radish 😤🐴", '@united so our flight into ord was delayed because of air force one, but the last flight to sbn is at 8:20, 5 mins from now we just landed.']
ytrain ['negative' 'positive' 'positive' 'negative' 'negative']


In [167]:
ytrain.shape

(10980,)

In [168]:
import string
punctuations = list(string.punctuation)
punctuations

['!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 '{',
 '|',
 '}',
 '~']

In [169]:
from sklearn.feature_extraction import text 
stop_words = text.ENGLISH_STOP_WORDS.union(punctuations)
stop_words                                                       #nltk should be download

frozenset({'!',
           '"',
           '#',
           '$',
           '%',
           '&',
           "'",
           '(',
           ')',
           '*',
           '+',
           ',',
           '-',
           '.',
           '/',
           ':',
           ';',
           '<',
           '=',
           '>',
           '?',
           '@',
           '[',
           '\\',
           ']',
           '^',
           '_',
           '`',
           'a',
           'about',
           'above',
           'across',
           'after',
           'afterwards',
           'again',
           'against',
           'all',
           'almost',
           'alone',
           'along',
           'already',
           'also',
           'although',
           'always',
           'am',
           'among',
           'amongst',
           'amoungst',
           'amount',
           'an',
           'and',
           'another',
           'any',
           'anyhow',
           'anyone',
   

In [170]:
xtest=xtest.text.values

In [172]:
from sklearn.feature_extraction.text import CountVectorizer

In [217]:
count_vec = CountVectorizer(max_features=1000,stop_words = stop_words)

In [218]:
x_train = count_vec.fit_transform(data)

In [219]:
x_train

<10980x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 75174 stored elements in Compressed Sparse Row format>

In [220]:
count_vec.get_feature_names()

['000',
 '10',
 '100',
 '11',
 '12',
 '15',
 '16',
 '17',
 '18',
 '19',
 '1hr',
 '1k',
 '1st',
 '20',
 '200',
 '2015',
 '21',
 '22',
 '23',
 '24',
 '25',
 '27',
 '2hrs',
 '2nd',
 '30',
 '35',
 '3rd',
 '40',
 '45',
 '50',
 '60',
 '728',
 '75',
 '800',
 '90',
 'aa',
 'able',
 'absolute',
 'absolutely',
 'acceptable',
 'access',
 'account',
 'actual',
 'actually',
 'add',
 'added',
 'address',
 'advisory',
 'afternoon',
 'agent',
 'agents',
 'ago',
 'air',
 'aircraft',
 'airline',
 'airlines',
 'airport',
 'airports',
 'airways',
 'allow',
 'allowed',
 'amazing',
 'america',
 'american',
 'americanair',
 'americanairlines',
 'amp',
 'angry',
 'answer',
 'answered',
 'answering',
 'answers',
 'anymore',
 'apology',
 'app',
 'apparently',
 'appreciate',
 'appreciated',
 'area',
 'aren',
 'arrival',
 'arrive',
 'arrived',
 'arriving',
 'asap',
 'ask',
 'asked',
 'asking',
 'assist',
 'assistance',
 'atl',
 'atlanta',
 'attendant',
 'attendants',
 'attitude',
 'austin',
 'auto',
 'automated',

In [221]:
x_train.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [222]:
xtest

array(["@AmericanAir In car gng to DFW. Pulled over 1hr ago - very icy roads. On-hold with AA since 1hr. Can't reach arpt for AA2450. Wat 2 do?",
       '@AmericanAir after all, the plane didn’t land in identical or worse) conditions at GRK according to METARs.',
       "@SouthwestAir can't believe how many paying customers you left high and dry with no reason for flight Cancelled Flightlations Monday out of BDL! Wow.",
       ...,
       'Nice RT @VirginAmerica: The man of steel might be faster, but we have WiFi – just saying. #ScienceBehindTheExperience http://t.co/FGRbpAZSiX',
       '@AmericanAir Aww Thanks AA..DFW was on GMA up here this AM..so i understand ..Btw A.A is my Airline when im able to trv..Love you guys.:)',
       '@united the lounge tells us they have no pillows for my grandma as one of the ladies opens the closet and I see 2 right there. #unitedlies'],
      dtype=object)

In [223]:
x_test= count_vec.transform(xtest)

In [224]:
from sklearn.svm import SVC
svc = SVC()
svc.fit(x_train,ytrain)
svc.score(x_train,ytrain)

C:\Users\hp\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.6253187613843352

In [225]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(x_train,ytrain)
ypred = rfc.predict(x_test)
print(rfc.score(x_train,ytrain))
print(rfc.score(x_test,ypred))
np.savetxt(r"C:\Users\hp\Downloads\twitter_csv.csv",ypred,fmt = "%s", delimiter=",")

C:\Users\hp\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.972040072859745
1.0


In [226]:
from sklearn.linear_model import LogisticRegression
clf=LogisticRegression(C=1)
clf.fit(x_train,ytrain)
clf.score(x_train,ytrain)

C:\Users\hp\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\hp\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.8254098360655737